In [1]:
import pandas as pd 
import scipy.stats as st
import numpy as np


# Mostrar todo el contenido de las celdas (sin recortes)
pd.set_option('display.max_colwidth', None)

# Mostrar todas las columnas 
pd.set_option('display.max_columns', None)

In [2]:
ruta = "/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv"

dp = pd.read_csv(ruta)

In [3]:
dp = dp.drop(columns=["Unnamed: 0"])

dp["store_name"] = dp["store_name"].replace({"PRINCIPAL COTA":"MOSTRADOR COTA"})

In [4]:
stock_seguridad = {
    "SUCURSAL CALLE 6": 50.00,
    "SUCURSAL VALLADOLID": 50.00,
    "SUCURSAL NORTE": 50.00,
    "SUCURSAL MEDELLIN": 150.00,
    "MOSTRADOR COTA": 300.00,
    "SUCURSAL BARRANQUILLA": 150.00,
    "SUCURSAL BUCARAMANGA": 150.00,
    "SUCURSAL CALI": 150.00,
   # "ANTIOQUEÑA DE LUBRICANTES SGP SAS": 300.00
}

reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    #"ANTIOQUEÑA DE LUBRICANTES SGP SAS": {"Punto de reorden": 5.00, "Inventario ideal": 7.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00}, # Tiene para 7 semnaas de inventario , y cubre el inventario 9 semanas
}


In [5]:
dp.head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
0,MOSTRADOR COTA,BCE00606125,2025.0,32,27.358720,20564.31,562613.20,0.0033,0.0497,6.656992e+11,1183227.14,0.01,0.000000,1183227.14,0.0,0.02,AAA,0.999
1,MOSTRADOR COTA,BCE00606125,2025.0,33,21.886976,20564.31,450090.56,0.0026,0.0727,1.665335e+05,0.37,0.01,4.000000,1183227.51,1.0,0.25,AAA,0.999
2,MOSTRADOR COTA,DCS10536137,2025.0,42,21.416133,51663.09,1106423.61,0.0065,0.0065,5.421476e+05,0.49,0.01,6.701031,1183228.00,1.0,0.23,AAA,0.999
3,MOSTRADOR COTA,DCS10536137,2025.0,43,19.274520,51663.09,995781.26,0.0058,0.0123,3.883547e+05,0.39,0.02,5.668712,1183228.39,1.0,0.23,AAA,0.999
4,MOSTRADOR COTA,DCS10536137,2025.0,31,18.083800,51663.09,934264.99,0.0055,0.0178,1.317314e+06,1.41,0.02,2.000000,1183229.80,0.8,0.20,AAA,0.999
5,MOSTRADOR COTA,BCE00606125,2025.0,34,17.509581,20564.31,360072.45,0.0021,0.0819,1.620326e+05,0.45,0.02,5.000000,1183230.25,1.0,0.25,AAA,0.999
6,MOSTRADOR COTA,DCS10536137,2025.0,44,17.347068,51663.09,896203.14,0.0053,0.0231,3.943294e+05,0.44,0.03,6.073620,1183230.69,1.0,0.24,AAA,0.999
7,MOSTRADOR COTA,DCS10536137,2025.0,32,16.275420,51663.09,840838.49,0.0049,0.0280,2.438432e+05,0.29,0.03,2.941176,1183230.98,0.8,0.20,AAA,0.999
8,MOSTRADOR COTA,DAB14570025,2025.0,35,15.942981,10188.65,162437.45,0.0010,0.1623,3.086310e+04,0.19,0.03,2.769231,1183231.17,0.8,0.24,AAA,0.999
9,MOSTRADOR COTA,DAB02570025,2025.0,35,15.467542,13771.02,213003.83,0.0013,0.1224,3.621070e+04,0.17,0.04,2.153846,1183231.34,0.8,0.24,AAA,0.999


## INVENTARIO DE SEGURIDAD

In [6]:
# Copia del df original
df_std = dp.copy()

# Ordenar datos por tienda, producto, año y semana
df_std = df_std.sort_values(by=["store_name", "product_ref", "Año", "semana"])

# Calcular la desviación móvil (rolling) de EMA con ventana de 12 semanas
df_std["Desviacion_EMA_calc"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).std())
)

# Calcular también la media móvil del EMA (opcional, para validación)
df_std["Ema_media"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)

# === INVENTARIO IDEAL y PUNTO DE REORDEN ===
# Diccionario con los dos parámetros (en semanas de cobertura)
reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
}

# Extraer columnas de reorden e ideal
df_std["lt_reorden"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Punto de reorden", np.nan))
df_std["lt_ideal"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Inventario ideal", np.nan))


# Calcular Z-score según nivel de servicio
df_std["Z"] = df_std["Nivel de servicio"].apply(lambda x: st.norm.ppf(x))

# === STOCK DE SEGURIDAD (rolling 12 semanas) ===
df_std["inventario_seguridad"] = (
    df_std["Z"] * df_std["Desviacion_EMA_calc"] * np.sqrt(df_std["lt_ideal"])
)

# Inventario ideal (nivel máximo recomendado)
df_std["inventario_ideal"] = (df_std["Ema"] * df_std["lt_ideal"]) + df_std["inventario_seguridad"]


# Punto de reorden (cuándo pedir)
df_std["punto_reorden"] = (df_std["Ema"] * df_std["lt_reorden"]) + df_std["inventario_seguridad"]



# === Redondear columnas numéricas ===
df_std["inventario_seguridad"] = df_std["inventario_seguridad"].round(2)
df_std["inventario_ideal"] = df_std["inventario_ideal"].round(2)
df_std["punto_reorden"] = df_std["punto_reorden"].round(2)




df_std = df_std.drop(columns=["Desviacion_EMA_calc","Ema_media","lt_reorden","lt_ideal","Z"])

# TARDA 2 MINUTOS 

In [7]:
df_std[(df_std["semana"]==43) & (df_std["store_name"]=="SUCURSAL BARRANQUILLA")].sort_values(by=["Ema"],ascending=False).head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio,inventario_seguridad,inventario_ideal,punto_reorden
53777,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,43,40.852424,13771.02,562579.55,0.0006,0.1718,106890.1,0.19,0.03,3.929448,3.74,1.0,0.31,AAA,0.999,73.41,441.08,359.37
53783,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,43,37.769983,10188.65,384825.14,0.0004,0.2737,76965.0,0.20,0.04,3.475352,4.93,1.0,0.34,AAA,0.999,56.84,396.77,321.23
53817,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,43,29.579996,32494.89,961198.72,0.0010,0.0949,105731.9,0.11,0.08,1.828829,8.97,0.8,0.26,AAA,0.999,35.28,301.50,242.34
53829,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,43,27.297403,20581.34,561817.13,0.0006,0.1730,73036.2,0.13,0.09,2.901205,10.40,0.8,0.30,AAA,0.999,30.49,276.16,221.57
53837,SUCURSAL BARRANQUILLA,DAC00037189,2025.0,43,26.276446,5315.94,139684.01,0.0001,0.5520,50286.2,0.36,0.09,4.783333,2031177.99,1.0,0.47,AAA,0.999,49.67,286.16,233.60
53841,SUCURSAL BARRANQUILLA,BCS00025125,2025.0,43,25.838374,31057.73,802481.24,0.0008,0.1031,104322.6,0.13,0.10,1.983003,2031178.21,0.8,0.28,AAA,0.999,31.02,263.57,211.89
53859,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,43,23.444745,10883.85,255169.09,0.0003,0.3787,45930.4,0.18,0.11,3.668269,2031180.06,1.0,0.43,AAA,0.999,48.71,259.72,212.83
53868,SUCURSAL BARRANQUILLA,BCS00249125,2025.0,43,22.812714,27389.88,624837.50,0.0007,0.1507,93725.6,0.15,0.12,2.972603,2031180.82,0.8,0.30,AAA,0.999,31.60,236.91,191.28
53879,SUCURSAL BARRANQUILLA,BCS00060125,2025.0,43,21.933249,30384.06,666421.15,0.0007,0.1311,126620.0,0.19,0.13,2.681992,2031182.10,0.8,0.30,AAA,0.999,35.11,232.51,188.64
53917,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,43,19.579354,11624.90,227608.03,0.0002,0.4038,18208.6,0.08,0.16,1.809231,2031187.03,0.8,0.41,AAA,0.999,18.10,194.31,155.15


# INVENTARIO ACTUAL

In [8]:
ruta = "/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx"

inv = pd.read_excel(ruta)

inv["product_ref"] = inv["Complete name"].str.extract(r"\[([A-Z0-9]+)\]")

In [9]:
inv.head(10)

,Sucursal,Estado,Codigo_interno,Complete name,Clase,Marca,categoria_producto,Cantidad,Costo unitario,Costo total,product_ref
0,PRINCIPAL COTA,Existencias,004 DA4207,[DAE04207004] DA4207 FILTRO AIRE SEGURIDAD KOMATSU (004 AF574SY),C,AUT*PARTS,Filtro,1.0,1.00,1.00,DAE04207004
1,PRINCIPAL COTA,Existencias,004 G314,[DLE00314004] G314 FILTRO ACEITE ESPECIAL (004 F62H),C,AUT*PARTS,Filtro,2.0,1890.00,3780.00,DLE00314004
2,PRINCIPAL COTA,Existencias,004 G401,[DLE00401004] G401 FILTRO ACEITE FIAT-ALLIS 75215486 (004 F75215486),C,AUT*PARTS,Filtro,7.0,1.00,7.00,DLE00401004
3,SUCURSAL MEDELLIN,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,2.0,1694.96,3389.92,DAE05128004
4,PRINCIPAL COTA,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,1.0,1694.96,1694.96,DAE05128004
5,PRINCIPAL COTA,Existencias,004 GA158,[DAE05158004] GA158 FILTRO AIRE RENULT 4 Y 6. (004 GA158),C,AUT*PARTS,Filtro,1.0,1722.50,1722.50,DAE05158004
6,PRINCIPAL COTA,Existencias,004 GA182,[DAE05182004] GA182 FILTRO AIRE SIMCA 1000 Y 1300 (004 GA182),C,AUT*PARTS,Filtro,1.0,1462.50,1462.50,DAE05182004
7,PRINCIPAL COTA,Existencias,004 GA321,[DAE05321004] GA321 FILTRO AIRE MONZA (004 GA321),C,AUT*PARTS,Filtro,2.0,1716.00,3432.00,DAE05321004
8,SUCURSAL NORTE,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004
9,SUCURSAL BUCARAGAMNGA,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004


In [10]:
inv["Cantidad"].sum() 

np.float64(306672.774)

# EVALUACION Y COMPARACION

In [11]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","punto_reorden","clasificacion","almacen_id","inventario_seguridad","inventario_ideal" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df.rename(columns={"product_name": "product_ref"}, inplace=True)


df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)


In [12]:
df_41 = df_std[df_std["semana"]==44]

df_41["Clasificacion_mia"] = df_41["Clasificacion"]

/tmp/ipykernel_1574/743102113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_41["Clasificacion_mia"] = df_41["Clasificacion"]


In [13]:
# --- MERGE LIMPIO PARA COMPARAR ---
df_merge_simple = pd.merge(
    df_41[['store_name', 'product_ref', 'Clasificacion_mia', 
            'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    df[['store_name', 'product_ref', 'clasificacion', 
         'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    on=['store_name', 'product_ref'],
    how='inner',
    suffixes=('_std', '_real')
)

# --- COMPARACIÓN DE CLASIFICACIÓN ---
df_merge_simple['match_clasificacion'] = (
    df_merge_simple['Clasificacion_mia'] == df_merge_simple['clasificacion']
)

# --- OPCIONAL: ORDENAR PARA REVISAR FÁCIL ---
df_merge_simple = df_merge_simple.sort_values(by=['store_name', 'product_ref'])

# --- MOSTRAR SOLO LAS COLUMNAS RELEVANTES ---
cols = [
    'store_name', 'product_ref',
    'Clasificacion_mia', 'clasificacion', 'match_clasificacion',
    'inventario_seguridad_std', 'inventario_seguridad_real',
    'inventario_ideal_std', 'inventario_ideal_real',
    'punto_reorden_std', 'punto_reorden_real'
]

df_merge_simple = df_merge_simple[cols]



df_merge_simple.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/diferencias_ema.xlsx")

In [14]:
df_merge_simple.sample(10)

,store_name,product_ref,Clasificacion_mia,clasificacion,match_clasificacion,inventario_seguridad_std,inventario_seguridad_real,inventario_ideal_std,inventario_ideal_real,punto_reorden_std,punto_reorden_real
21781,SUCURSAL NORTE,DAB02974025,C,C,True,0.0,0.0,0.00,0,0.00,0
13662,SUCURSAL MEDELLIN,BAB08057125,C,C,True,0.0,0.0,0.00,0,0.00,0
25909,SUCURSAL VALLADOLID,DAE06992010,C,C,True,0.0,0.0,0.00,1,0.00,0
2383,SUCURSAL BARRANQUILLA,DAE06390024,C,C,True,0.0,0.0,0.00,1,0.00,0
6654,SUCURSAL BUCARAMANGA,DLS00232189,C,C,True,0.0,0.0,1.17,2,0.91,2
10372,SUCURSAL CALLE 6,BAE02553125,C,C,True,0.0,0.0,0.00,0,0.00,0
14448,SUCURSAL MEDELLIN,BAX00122125,C,C,True,0.0,0.0,0.00,0,0.00,0
13330,SUCURSAL CALLE 6,DLS00010010,C,C,True,0.0,0.0,0.00,0,0.00,0
2779,SUCURSAL BARRANQUILLA,DAX12671025,C,C,True,0.0,0.0,0.00,1,0.00,1
229,SUCURSAL BARRANQUILLA,BAC00215125,C,C,True,0.0,0.0,0.00,0,0.00,0
